In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [TODO] Add your H1 title heading here

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.13

## Overview

{TODO: Include a paragraph or two explaining what this example demonstrates, who should be interested in it, and what you need to know before you get started.}

Learn more about [web-doc-title](linkback-to-webdoc-page). {TODO: if more than one primary feature, add tag/linkback for each one}

### Objective

In this tutorial, you learn how to {TODO: Complete the sentence explaining briefly what you will learn from the notebook, such as
training, hyperparameter tuning, or serving}:

This tutorial uses the following Google Cloud ML services and resources:

- *{TODO: Add high level bullets for the services/resources demonstrated; e.g., Vertex AI Training}*


The steps performed include:

- *{TODO: Add high level bullets for the steps of performed in the notebook}*

### Dataset

{TODO: Include a paragraph with Dataset information and where to obtain it.} 

{TODO: Make sure the dataset is accessible to the public. **Googlers**: Add your dataset to the [public samples bucket](http://goto/cloudsamples#sample-storage-bucket) within gs://cloud-samples-data/vertex-ai, if it doesn't already exist there.}

### Costs 

{TODO: Update the list of billable products that your tutorial uses.}

This tutorial uses billable components of Google Cloud:

* Vertex AI
* {TODO: BigQuery}
* Cloud Storage

{TODO: Include links to pricing documentation for each product you listed above.
 NOTE: If you use BigQuery or Dataflow, you need to add this to the pricing.
}

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
{ TODO: [BigQuery pricing](https://cloud.google.com/bigquery/pricing), }
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing), 
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

{TODO: Suggest using the latest major GA version of each package; i.e., --upgrade}

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [1]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

The Cloud SDK, code and other libraries currently run as the service account identity of the Workbench Instance running this notebook.

**- Authenticate the Cloud SDK with your credentials :**

In [2]:
# ! gcloud auth login

**- Authenticate code and libraries with your credentials :**

In [ ]:
# ! gcloud auth application-default

**- Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

- *{Note to notebook author: For any user-provided strings that need to be unique (like bucket names or model ID's), append "-unique" to the end so proper testing can occur}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
from google.cloud import aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

{TODO: Include commands to delete individual resources below}

In [1]:
import os

# Delete endpoint resource
# e.g. `endpoint.delete()`

# Delete model resource
# e.g. `model.delete()`

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI

# What you will learn:
Query a public dataset using BigQuery.
Convert a BigQuery dataset to embeddings using the Vertex AI Text-Embeddings API.
Create an index in Vertex AI Vector Search.
Upload embeddings to the index.
Create an index endpoint.
Deploy the index to the index endpoint.
Perform an online query.

In the first cell, run the following command to install the Google Cloud Vertex AI, Cloud Storage and BigQuery SDKs. To run the command, execute SHIFT+ENTER.

In [1]:
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]'

  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.14.0
    Uninstalling google-cloud-storage-2.14.0:
      Successfully uninstalled google-cloud-storage-2.14.0


In [3]:
# restart kernal
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [1]:
# setting up the enivironment values for the project

PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east1"

In [2]:
import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION)

### Task 4. Prepare the data in BigQuery
The dataset used for this lab is the StackOverflow dataset. This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

Stack Overflow is the largest online community for programmers to learn, share their knowledge, and advance their careers. Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

The BigQuery table is too large to fit into memory, so you need to write a generator called query_bigquery_chunks to yield chunks of the dataframe for processing. Additionally, an extra column title_with_body is added, which is a concatenation of the question title and body that will be used for creating embeddings.

In [3]:
# Import the libraries and initialize the BigQuery client.
import math
from typing import Any, Generator

import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

In [4]:
# Define the BigQuery query for the remote dataset.
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title, q.body
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q
        LIMIT {limit} OFFSET {offset};
        """

This code defines a generator function called query_bigquery_chunks that is used to query a BigQuery database in chunks. Here's a breakdown of what the code does:

Function signature

The function takes three parameters:

max_rows: The maximum number of rows to query from the database.
rows_per_chunk: The number of rows to query in each chunk.
start_chunk: The starting chunk number (default is 0).
The function returns a generator that yields pandas DataFrames.

Looping over chunks

The function uses a for loop to iterate over the chunks of rows to query. The loop starts at start_chunk and increments by rows_per_chunk until it reaches max_rows.

Constructing the query

Inside the loop, the function constructs a query string using the QUERY_TEMPLATE format string. The query string includes the limit and offset parameters, which are set to rows_per_chunk and offset, respectively.

Running the query

The function uses the client.query method to run the query and retrieve the results.

Processing the results

The function processes the results by converting them to a pandas DataFrame using the to_dataframe method. It then adds a new column called title_with_body by concatenating the title and body columns with a newline character (\n) in between.

Yielding the DataFrame

Finally, the function yields the processed DataFrame.

Example usage

Here's an example of how you might use this function:

In [6]:
def query_bigquery_chunks(
    max_rows: int, rows_per_chunk: int, start_chunk: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        yield df

In [7]:
df = next(query_bigquery_chunks(max_rows=1000, rows_per_chunk=1000))

# Examine the data
df.head()



,id,title,body,title_with_body
0,4255120,How to revert an HttpError ConfigurationElemen...,<p>Suppose I have the following <code>&lt;http...,How to revert an HttpError ConfigurationElemen...
1,4421341,Haskell data types usage good practicies,"<p>Reading ""Real world Haskell"" i found some i...",Haskell data types usage good practicies\n<p>R...
2,4492944,Extract base url from full url,<p>Any quick way to extract base url from full...,Extract base url from full url\n<p>Any quick w...
3,4473573,Android Landscape mode cutting off my buttons ...,<p>I am having an issue with my application. I...,Android Landscape mode cutting off my buttons ...
4,4588542,Sorting NSArray which many contain number,<p>I have an NSArray which is populated with o...,Sorting NSArray which many contain number\n<p>...


In [9]:
df['title'][0]

'How to revert an HttpError ConfigurationElement back to "Inherited" using IIS 7 API'

In [10]:
df['body'][0]

'<p>Suppose I have the following <code>&lt;httpErrors&gt;</code> collection in a <code>web.config</code>:</p>\n\n<pre><code>&lt;httpErrors&gt;\n&lt;/httpErrors&gt;\n</code></pre>\n\n<p>Yep, nice \'n empty.</p>\n\n<p>And in IIS 7, my HTTP errors page looks like this:</p>\n\n<p><img src="https://i.stack.imgur.com/EBcgJ.png" alt="httpErrors"></p>\n\n<p>Beautiful! (I\'ve highlighted 404 simply because that\'s the example I\'ll use in a sec).</p>\n\n<p>Now, I run the following code:</p>\n\n<pre><code>errorElement["statusCode"] = 404;\nerrorElement["subStatusCode"] = -1;\nerrorElement["path"] = "/404.html";\nhttpErrorsCollection.Add(errorElement);\n</code></pre>\n\n<p>Lovely. I now have, as expected this in my <code>web.config</code>:</p>\n\n<pre><code>&lt;httpErrors&gt;\n    &lt;remove statusCode="404" subStatusCode="-1" /&gt;\n    &lt;error statusCode="404" subStatusCode="-1" prefixLanguageFilePath="" path="/404.html" /&gt;\n&lt;/httpErrors&gt;\n</code></pre>\n\n<p>Couldn\'t be happier. No

# Task 5. Create text embeddings from BigQuery data

In [11]:
from typing import List, Optional
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("text-embedding-004")

This code defines a function called encode_texts_to_embeddings that takes a list of sentences as input and returns a list of embeddings for each sentence. Here's a breakdown of what the code does:

Function signature

The function takes one parameter:

sentences: A list of strings, where each string is a sentence.
The function returns a list of optional lists of floats, where each inner list represents the embedding for a sentence.

Try-except block

The function uses a try-except block to handle any exceptions that may occur during the execution of the code.

Getting embeddings

Inside the try block, the function calls the get_embeddings method of the model object, passing in the list of sentences as input. This method is expected to return a list of embeddings, where each embedding is a vector representation of a sentence.

Processing embeddings

The function then processes the embeddings by extracting the values from each embedding object and storing them in a list. This is done using a list comprehension: [embedding.values for embedding in embeddings].

Returning embeddings

The function returns the list of embeddings.

Handling exceptions

If an exception occurs during the execution of the code, the function catches the exception and returns a list of None values, where each None value corresponds to a sentence in the input list. This is done using a list comprehension: [None for _ in range(len(sentences))].

In [13]:
# Define an embedding method that uses the model.
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

According to the documentation, each request can handle up to 5 text instances. So we will need to split the BigQuery question results in batches of 5 before sending to the embedding API.

Create a generate_batches to split results in batches of 5 to be sent to the embeddings API.

This code defines a generator function called generate_batches that takes a list of sentences and a batch size as input and yields batches of sentences. Here's a breakdown of what the code does:

Function signature

The function takes two parameters:

sentences: A list of strings, where each string is a sentence.
batch_size: An integer that specifies the number of sentences to include in each batch.
The function returns a generator that yields lists of strings, where each list represents a batch of sentences.

Generator function

The function uses a generator function to yield batches of sentences. A generator function is a special type of function that can be used to generate a sequence of values on the fly, rather than computing them all at once and storing them in memory.

Looping over sentences

The function uses a for loop to iterate over the list of sentences in batches of batch_size. The loop starts at index 0 and increments by batch_size each time.

Yielding batches

Inside the loop, the function yields a batch of sentences using the yield keyword. The batch is created by slicing the list of sentences from the current index i to i + batch_size.

In [16]:
import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple

import numpy as np
from tqdm.auto import tqdm


# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

Encapsulate the process of generating batches and calling the embeddings API in a method called encode_text_to_embedding_batched. This method also handles rate-limiting using time.sleep. For production use cases, you would want a more sophisticated rate-limiting mechanism that takes retries into account.

In [17]:
# def encode_text_to_embedding_batched(
#     sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
# ) -> Tuple[List[bool], np.ndarray]:

#     embeddings_list: List[List[float]] = []

#     # Prepare the batches using a generator
#     batches = generate_batches(sentences, batch_size)

#     seconds_per_job = 1 / api_calls_per_second

#     with ThreadPoolExecutor() as executor:
#         futures = []
#         for batch in tqdm(
#             batches, total=math.ceil(len(sentences) / batch_size), position=0
#         ):
#             futures.append(
#                 executor.submit(functools.partial(encode_texts_to_embeddings), batch)
#             )
#             time.sleep(seconds_per_job)

#         for future in futures:
#             embeddings_list.extend(future.result())

#     is_successful = [
#         embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
#     ]
#     embeddings_list_successful = np.squeeze(
#         np.stack([embedding for embedding in embeddings_list if embedding is not None])
#     )
#     return is_successful, embeddings_list_successful

Test the encoding function by encoding a subset of data and see if the embeddings and distance metrics make sense.

In [24]:
def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(executor.submit(encode_texts_to_embeddings, batch))
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [embedding is not None for embedding in embeddings_list]
    embeddings_list_successful = np.array(
        [embedding for embedding in embeddings_list if embedding is not None]
    )
    return is_successful, embeddings_list_successful


In [18]:
# Encode a subset of questions for validation
questions = df.title.tolist()[:500]
is_successful, question_embeddings = encode_text_to_embedding_batched(
    sentences=df.title.tolist()[:500]
)

# Filter for successfully embedded sentences
questions = np.array(questions)[is_successful]

  0%|          | 0/100 [00:00<?, ?it/s]

Save the dimension size for later usage when creating the Vertex AI Vector Search index.

In [19]:
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

768


Sort questions in order of similarity. According to the embedding documentation, the similarity of embeddings is calculated using the dot-product, with np.dot. Once you have the similarity score, sort the results and print them for inspection. 1 means very similar, 0 means very different.

In [20]:
import random

question_index = random.randint(0, 99)

print(f"Query question = {questions[question_index]}")

# Get similarity scores for each embedding by using dot-product.
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

Query question = Finding All Combinations (Cartesian product) of JavaScript array values
	0: Finding All Combinations (Cartesian product) of JavaScript array values: 0.9999977794032682
	1: Javascript, filter object list using string condition?: 0.5625301903976244
	2: Match backwards from a given word with javascript: 0.5504823359661402
	3: javascript chinese/japanese character decoding: 0.5147855421339054
	4: PHP recursion: How to create a recursion in php: 0.4875655962986394
	5: How can I populate a select dropdown list from a JSON feed with AngularJS?: 0.4775828568939623
	6: how to extract this array through foreach loop: 0.47443788106366486
	7: scala: accumulate a var from collection in a functional manner (that is, no vars): 0.47109930942906036
	8: jQuery & document.write: 0.46812234735227976
	9: Difference between floats and ints in Javascript?: 0.4484692176295221
	10: How to execute JS from Python, that uses 'Document' and/or 'Window': 0.4429945344773859
	11: How to Convert an Ar

### Save the embeddings in JSONL format. The data must be formatted in JSONL format, which means each embedding dictionary is written as an individual JSON object on its own line.

In [22]:
import tempfile
from pathlib import Path

# Create temporary file to write embeddings to
embeddings_file_path = Path(tempfile.mkdtemp())

print(f"Embeddings directory: {embeddings_file_path}")

Embeddings directory: /var/tmp/tmp9fm69ik5


### Write embeddings in batches to prevent out-of-memory errors. Notice we are only using 5000 questions so that the embedding creation process and indexing is faster. The dataset contains more than 50,000 questions. This step will take around 5 minutes.

This code is designed to process a large dataset from BigQuery in chunks, convert each chunk to embeddings using an API, and save the embeddings to a file. Here's a breakdown of what the code does:

**Constants and variables**

The code defines several constants and variables:

* `BQ_NUM_ROWS`: The total number of rows to process from BigQuery.
* `BQ_CHUNK_SIZE`: The number of rows to process in each chunk.
* `BQ_NUM_CHUNKS`: The total number of chunks to process, calculated by dividing `BQ_NUM_ROWS` by `BQ_CHUNK_SIZE`.
* `START_CHUNK`: The starting chunk number, set to 0.
* `API_CALLS_PER_SECOND`: The rate limit for API calls, set to 300 requests per minute.
* `ITEMS_PER_REQUEST`: The number of items that can be processed per API request, set to 5.

**Looping through chunks**

The code uses a `for` loop to iterate through each chunk of data from BigQuery. The loop uses the `query_bigquery_chunks` function to generate each chunk, and the `tqdm` library to display a progress bar.

**Processing each chunk**

Inside the loop, the code processes each chunk as follows:

1. Creates a unique output file for each chunk using the `embeddings_file_path` variable.
2. Extracts the `id` column from the chunk using the `id_chunk` variable.
3. Converts the chunk to embeddings using the `encode_text_to_embedding_batched` function, passing in the `sentences` column from the chunk, the `api_calls_per_second` rate limit, and the `batch_size` of 5 items per request.
4. Formats the embeddings as a list of JSON objects, where each object contains the `id` and `embedding` values.
5. Appends the formatted embeddings to the output file using the `writelines` method.
6. Deletes the chunk DataFrame and any other large data structures using the `del` statement and the `gc.collect` function to free up memory.

**Saving embeddings to file**

The code saves the embeddings to a file in JSON format, where each line represents a single embedding. The file is created in the same directory as the `embeddings_file_path` variable, with a unique name for each chunk.

Overall, this code is designed to process a large dataset from BigQuery in chunks, convert each chunk to embeddings using an API, and save the embeddings to a file in JSON format.

In [25]:
import gc
import json

BQ_NUM_ROWS = 5000
BQ_CHUNK_SIZE = 1000
BQ_NUM_CHUNKS = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)

START_CHUNK = 0

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 300 / 60
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 5

# Loop through each generated dataframe, convert
for i, df in tqdm(
    enumerate(
        query_bigquery_chunks(
            max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=START_CHUNK
        )
    ),
    total=BQ_NUM_CHUNKS - START_CHUNK,
    position=-1,
    desc="Chunk of rows from BigQuery",
):
    # Create a unique output file for each chunk
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{i+START_CHUNK}.json"
    )
    with open(chunk_path, "a") as f:
        id_chunk = df.id

        # Convert batch to embeddings
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
            sentences=df.title_with_body.to_list(),
            api_calls_per_second=API_CALLS_PER_SECOND,
            batch_size=ITEMS_PER_REQUEST,
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

        # Delete the DataFrame and any other large data structures
        del df
        gc.collect()

Chunk of rows from BigQuery:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

## Task 6. Upload embeddings to Cloud Storage
Upload the text-embeddings to Cloud Storage, so that Vertex AI Vector Search can access them later.

In [28]:
#Define a bucket where you will store your embeddings.
BUCKET_URI = f"gs://{PROJECT_ID}-unique"

In [29]:
#Create your Cloud Storage bucket.
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://qwiklabs-gcp-01-aac69c1b7459-unique/...


In [30]:
# Upload the training data to a Google Cloud Storage bucket.
remote_folder = f"{BUCKET_URI}/{embeddings_file_path.stem}/"
! gsutil -m cp -r {embeddings_file_path}/* {remote_folder}

Copying file:///var/tmp/tmp9fm69ik5/tmp9fm69ik5_1.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp9fm69ik5/tmp9fm69ik5_0.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp9fm69ik5/tmp9fm69ik5_3.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp9fm69ik5/tmp9fm69ik5_2.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp9fm69ik5/tmp9fm69ik5_4.json [Content-Type=application/json]...
/ [5/5 files][ 12.3 MiB/ 12.3 MiB] 100% Done                                    
Operation completed over 5 objects/12.3 MiB.                                     


### Task 7. Create an Index in Vertex AI Vector Search for your embeddings

In [31]:
# Setup your index name and description.
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "question titles and bodies from stackoverflow"

Create the index. Notice that the index reads the embeddings from the Cloud Storage bucket. The indexing process can take from 45 minutes up to 60 minutes. Wait for completion, and then proceed. You can open a different Google Cloud Console page, navigate to Vertex AI Vector search, and see how the index is being created.

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

DIMENSIONS = 768

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

## Reference the index name to make sure it got created successfully.

In [37]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/80366348363/locations/us-east1/indexes/2554007181649248256'

In [ ]:
## Using the resource name, you can retrieve an existing MatchingEngineIndex.

In [38]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

In [ ]:
# Create an IndexEndpoint so that it can be accessed via an API.

In [39]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/80366348363/locations/us-east1/indexEndpoints/5249816178909511680/operations/6460950914332098560
MatchingEngineIndexEndpoint created. Resource name: projects/80366348363/locations/us-east1/indexEndpoints/5249816178909511680
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/80366348363/locations/us-east1/indexEndpoints/5249816178909511680')


In [ ]:
# Deploy your index to the created endpoint. This can take up to 15 minutes.
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

DEPLOYED_INDEX_ID


my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/80366348363/locations/us-east1/indexEndpoints/5249816178909511680
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/80366348363/locations/us-east1/indexEndpoints/5249816178909511680/operations/1214257348445470720


Verify number of declared items matches the number of embeddings. Each IndexEndpoint can have multiple indexes deployed to it. For each index, you can retrieve the number of deployed vectors using the index_endpoint._gca_resource.index_stats.vectors_count. The numbers may not match exactly due to potential rate-limiting failures incurred when using the embedding service.

In [ ]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Expected: {BQ_NUM_ROWS}, Actual: {number_of_vectors}")

Task 8. Create online queries
After you build your indexes, you may query against the deployed index to find nearest neighbors.

Note: For the DOT_PRODUCT_DISTANCE distance type, the "distance" property returned with each MatchNeighbor actually refers to the similarity.

In [ ]:
# Create an embedding for a test question.
test_embeddings = encode_texts_to_embeddings(sentences=["Install GPU for Tensorflow"])

In [ ]:
# Test the query to retrieve the similar embeddings.
NUM_NEIGHBOURS = 10

response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

In [ ]:
# Verify that the retrieved results are relevant by checking the StackOverflow links.
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")